In [160]:
import numpy as np
import pandas as pd

In [161]:
df = pd.read_csv('Cleaned_Data.csv')
df.shape

(18832, 120)

# Select columns that affect performance of the model

## Remove columns with multicollinearity.

In [162]:

columns_to_remove = [
    'customer_id', 'SHORT_TERM_COUNT', 'MID_TERM_COUNT', 'LONG_TERM_COUNT',
    'NUMBER_OF_LOANS', 'NUMBER_OF_CREDIT_CARDS', 'NUMBER_OF_RELATIONSHIP',
    'OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_ALL_6M', 'OUTSTANDING_BAL_ALL_9M', 'OUTSTANDING_BAL_ALL_12M',
    'OUTSTANDING_BAL_LOAN_3M_6M', 'OUTSTANDING_BAL_LOAN_6M_9M', 'OUTSTANDING_BAL_LOAN_9M_12M',
    'OUTSTANDING_BAL_LOAN_6M_12M', 'OUTSTANDING_BAL_LOAN_3M_12M', 'OUTSTANDING_BAL_CC_3M_6M',
    'OUTSTANDING_BAL_CC_6M_9M', 'OUTSTANDING_BAL_CC_9M_12M', 'OUTSTANDING_BAL_CC_6M_12M',
    'OUTSTANDING_BAL_CC_3M_12M', 'OUTSTANDING_BAL_ALL_3M_6M', 'OUTSTANDING_BAL_ALL_6M_9M',
    'OUTSTANDING_BAL_ALL_9M_12M', 'OUTSTANDING_BAL_ALL_6M_12M', 'OUTSTANDING_BAL_ALL_3M_12M',
    'ENQUIRIES_3M_6M', 'ENQUIRIES_6M_9M', 'ENQUIRIES_9M_12M', 'ENQUIRIES_6M_12M',
    'ENQUIRIES_3M_12M', 'ENQUIRIES_FROM_BANK_3M_6M', 'ENQUIRIES_FROM_BANK_6M_9M',
    'ENQUIRIES_FROM_BANK_9M_12M', 'ENQUIRIES_FROM_BANK_6M_12M', 'ENQUIRIES_FROM_BANK_3M_12M',
    'ENQUIRIES_FROM_NON_BANK_3M_6M', 'ENQUIRIES_FROM_NON_BANK_6M_9M',
    'ENQUIRIES_FROM_NON_BANK_9M_12M', 'ENQUIRIES_FROM_NON_BANK_6M_12M',
    'ENQUIRIES_FROM_NON_BANK_3M_12M'
]

df = df.drop(columns_to_remove,axis=1)



## Remove outlier

In [163]:

columns_to_check = [
    'OUTSTANDING_BAL_LOAN_CURRENT', 'OUTSTANDING_BAL_LOAN_3M', 'OUTSTANDING_BAL_LOAN_6M', 'OUTSTANDING_BAL_LOAN_9M',
    'OUTSTANDING_BAL_LOAN_12M', 'OUTSTANDING_BAL_CC_3M', 'OUTSTANDING_BAL_CC_6M', 'OUTSTANDING_BAL_CC_9M',
    'OUTSTANDING_BAL_CC_12M', 'OUTSTANDING_BAL_ALL_CURRENT', 'SHORT_TERM_COUNT_BANK', 'MID_TERM_COUNT_BANK', 'LONG_TERM_COUNT_BANK',
    'SHORT_TERM_COUNT_NON_BANK', 'MID_TERM_COUNT_NON_BANK', 'LONG_TERM_COUNT_NON_BANK', 'NUMBER_OF_LOANS_BANK', 
    'NUMBER_OF_LOANS_NON_BANK', 'NUMBER_OF_CREDIT_CARDS_BANK', 'NUMBER_OF_CREDIT_CARDS_NON_BANK', 
    'NUMBER_OF_RELATIONSHIP_BANK', 'NUMBER_OF_RELATIONSHIP_NON_BANK', 'NUM_NEW_LOAN_TAKEN_BANK_3M', 
    'NUM_NEW_LOAN_TAKEN_BANK_6M', 'NUM_NEW_LOAN_TAKEN_BANK_9M', 'NUM_NEW_LOAN_TAKEN_BANK_12M', 
    'NUM_NEW_LOAN_TAKEN_NON_BANK_3M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_6M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_9M', 
    'NUM_NEW_LOAN_TAKEN_NON_BANK_12M'
]

def remove_outliers_iqr(df, columns):
    Q1 = df[columns].quantile(0.25)
    Q3 = df[columns].quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df[columns] < (Q1 - 1.5 * IQR)) |(df[columns] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out

df = remove_outliers_iqr(df, columns_to_check)


## Loại những cột có độ lệch chuẩn bằng không

In [164]:
def drop_zero_std_features(X):
    zero_std_cols = X.columns[X.std() == 0]
    if len(zero_std_cols) > 0:
        print(f"Loại bỏ các biến có độ lệch chuẩn bằng 0: {list(zero_std_cols)}")
        X = X.drop(columns=zero_std_cols)
    return X

df = drop_zero_std_features(df)

Loại bỏ các biến có độ lệch chuẩn bằng 0: ['MID_TERM_COUNT_BANK', 'LONG_TERM_COUNT_BANK', 'LONG_TERM_COUNT_NON_BANK']


## Loại bỏ những cột có tính tương quan cao

In [165]:

def remove_highly_correlated_features(X, correlation_threshold=0.95):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr_cols = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
    if high_corr_cols:
        print(f"Loại bỏ các biến có tương quan cao (> {correlation_threshold}):")
        print(high_corr_cols)
        X = X.drop(columns=high_corr_cols)
    return X
df = remove_highly_correlated_features(df)

Loại bỏ các biến có tương quan cao (> 0.95):
['NUMBER_OF_LOANS_BANK', 'NUM_NEW_LOAN_TAKEN_BANK_3M', 'NUM_NEW_LOAN_TAKEN_BANK_6M', 'NUM_NEW_LOAN_TAKEN_BANK_9M', 'NUM_NEW_LOAN_TAKEN_BANK_12M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_3M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_6M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_9M', 'NUM_NEW_LOAN_TAKEN_NON_BANK_12M', 'OUTSTANDING_BAL_LOAN_9M', 'OUTSTANDING_BAL_LOAN_12M', 'OUTSTANDING_BAL_CC_6M', 'OUTSTANDING_BAL_CC_9M', 'OUTSTANDING_BAL_CC_12M', 'ENQUIRIES_12M', 'ENQUIRIES_FOR_LOAN_3M', 'ENQUIRIES_FROM_BANK_FOR_LOAN_3M', 'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M', 'ENQUIRIES_FROM_BANK_FOR_CC_3M', 'ENQUIRIES_FOR_LOAN_6M', 'ENQUIRIES_FROM_BANK_FOR_LOAN_6M', 'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M', 'ENQUIRIES_FROM_BANK_FOR_CC_6M', 'ENQUIRIES_FOR_LOAN_9M', 'ENQUIRIES_FROM_BANK_FOR_LOAN_9M', 'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M', 'ENQUIRIES_FROM_BANK_FOR_CC_9M', 'ENQUIRIES_FROM_NON_BANK_12M', 'ENQUIRIES_FOR_LOAN_12M', 'ENQUIRIES_FOR_CC_12M', 'ENQUIRIES_FROM_BANK_FOR_LOAN_12M', 'ENQUIRIES_

In [166]:
df[['ENQUIRIES_3M','ENQUIRIES_6M','ENQUIRIES_9M']]

,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M
0,13.0,13.0,23.0
1,18.0,18.0,28.0
2,33.0,63.0,78.0
3,13.0,23.0,23.0
4,13.0,13.0,13.0
...,...,...,...
18822,18.0,18.0,18.0
18826,13.0,13.0,13.0
18827,13.0,13.0,13.0
18828,13.0,13.0,13.0


## Loại những cột có tính đa cộng tuyến 

In [167]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Hàm tính toán VIF
def calculate_vif(X):
    vif_df = pd.DataFrame()
    vif_df['feature'] = X.columns
    vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_df

# Hàm loại bỏ các biến có VIF cao
def remove_high_vif_features(X, threshold=5.0):
    while True:
        vif_df = calculate_vif(X)
        max_vif = vif_df['VIF'].max()
        if np.isinf(max_vif) or max_vif > threshold:
            if np.isinf(max_vif):
                max_vif_feature = vif_df.loc[vif_df['VIF'] == max_vif, 'feature'].values[0]
                print(f"Đa cộng tuyến hoàn hảo được phát hiện với biến '{max_vif_feature}'. Loại bỏ biến này.")
            else:
                max_vif_feature = vif_df.loc[vif_df['VIF'] == max_vif, 'feature'].values[0]
                print(f"Loại bỏ '{max_vif_feature}' do VIF cao ({max_vif:.2f})")
            X = X.drop(columns=[max_vif_feature])
        else:
            break
    return X


# Loại bỏ các biến có VIF cao
df = remove_high_vif_features(df)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Đa cộng tuyến hoàn hảo được phát hiện với biến 'ENQUIRIES_3M'. Loại bỏ biến này.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Đa cộng tuyến hoàn hảo được phát hiện với biến 'ENQUIRIES_6M'. Loại bỏ biến này.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Đa cộng tuyến hoàn hảo được phát hiện với biến 'ENQUIRIES_9M'. Loại bỏ biến này.
Loại bỏ 'OUTSTANDING_BAL_LOAN_CURRENT' do VIF cao (4595509823847.45)
Loại bỏ 'NUMBER_OF_LOANS_NON_BANK' do VIF cao (1848009695268.98)
Loại bỏ 'ENQUIRIES_FOR_CC_6M' do VIF cao (130.55)
Loại bỏ 'ENQUIRIES_FROM_BANK_9M' do VIF cao (49.08)
Loại bỏ 'NUMBER_OF_RELATIONSHIP_BANK' do VIF cao (26.71)
Loại bỏ 'ENQUIRIES_FROM_NON_BANK_FOR_CC_9M' do VIF cao (22.22)
Loại bỏ 'NUMBER_OF_RELATIONSHIP_NON_BANK' do VIF cao (19.54)
Loại bỏ 'ENQUIRIES_FOR_CC_3M' do VIF cao (19.33)
Loại bỏ 'ENQUIRIES_FROM_NON_BANK_6M' do VIF cao (16.06)
Loại bỏ 'OUTSTANDING_BAL_LOAN_3M' do VIF cao (10.74)
Loại bỏ 'ENQUIRIES_FOR_CC_9M' do VIF cao (10.49)
Loại bỏ 'NUM_NEW_LOAN_TAKEN_9M' do VIF cao (9.72)
Loại bỏ 'OUTSTANDING_BAL_ALL_CURRENT' do VIF cao (9.14)
Loại bỏ 'ENQUIRIES_FROM_BANK_6M' do VIF cao (6.40)
Loại bỏ 'ENQUIRIES_FROM_NON_BANK_FOR_CC_6M' do VIF cao (6.35)
Loại bỏ 'NUM_NEW_LOAN_TAKEN_12M' do VIF cao (5.48)


In [168]:
df.shape

(11417, 24)

In [176]:
df.to_csv('Model_Data.csv', index=False)

## Xay Dung Model

In [174]:
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
import joblib

# Assuming df is your DataFrame
X = df.drop(columns='label')
y = df['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
}

# Define the pipeline
estimator = Pipeline([
    ('smoteenn', SMOTEENN()),  # SMOTEENN for resampling
    ('scaler', StandardScaler()),  # Standardization
    ('classification', XGBClassifier(eval_metric='auc', use_label_encoder=False))  # XGBoost with AUC metric
])

# Expanded parameter grid for XGBoost
param_grid = {
    'smoteenn__sampling_strategy': [0.1, 0.3, 0.5, 0.7, 1.0],
    'classification__max_depth': [5, 7, 9, 15],  # Hạn chế max_depth
    'classification__n_estimators': [200, 300, 500],  # Hợp lý hóa n_estimators
    'classification__learning_rate': [0.001, 0.005, 0.01],  # Giữ lại các giá trị learning rate
    'classification__colsample_bytree': [0.7, 0.8, 1.0],
    'classification__subsample': [0.8, 1.0],
    'classification__min_child_weight': [1, 2, 3],  # Điều chỉnh min_child_weight hợp lý
    'classification__scale_pos_weight': [5, 10, 20, 30, 50, 100]  # Điều chỉnh scale_pos_weight
}

# Randomized search with cross-validation
random_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_grid,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    scoring=scoring,
    refit='f1',  # Refitting based on Recall to focus on improving it
    n_iter=50,  # Increased iterations for more extensive search
    n_jobs=-1,  # Use all available CPUs
    verbose=1
)

# Fit the random search on the training data
random_search.fit(X_train, y_train)

# Get the best model from random search
best_model = random_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Display the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Optionally save the best model
joblib.dump(best_model, 'best_xgb_model_tuned.pkl')


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [22:50:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [22:50:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [22:50:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [22:50:01] WARNING: /Users/runner/work/xgboost/xgboost/src


Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.91      1731
           1       0.83      0.54      0.65       553

    accuracy                           0.86      2284
   macro avg       0.85      0.75      0.78      2284
weighted avg       0.86      0.86      0.85      2284



['best_xgb_model_tuned.pkl']